## Homework 6 Visualization 1
### This notebook shows the initial plot we created to visualize the data as a part of Homework 6. 
**Note:** I used another notebook for this visualization because the graph is too big to show inside the notebook like the graphs in the original notebook.

In this visualization, We will focus on visualizing the user-wise edits on a timeline. We will color the edits red if bytes were removed and blue if bytes were added in the edit. The size of the circles represent the number of bytes changed.

In [5]:
import numpy as np
import pandas as pd
import re
from bokeh.plotting import figure
from bokeh.io import output_notebook, show,output_file,install_notebook_hook
from bokeh.models import ColumnDataSource, Select,LabelSet, HoverTool, DatetimeAxis, TapTool, CustomJS, BoxZoomTool, PanTool
from bokeh.models import WheelZoomTool, UndoTool, RedoTool, ResetTool, ZoomInTool, ZoomOutTool
import nltk
import re
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans,DBSCAN
import pprint
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from bokeh.models.widgets import DataTable, TableColumn, Div, RangeSlider, Slider, Select
from bokeh.layouts import Row,widgetbox, Column
from sklearn.manifold import MDS
from sklearn.decomposition import TruncatedSVD
from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application
from bokeh.palettes import Category10
import math

In [6]:
file = open('data/wiki sample data.txt','r', errors='ignore')

lines = file.readlines()
#print (lines)

df = pd.DataFrame(columns=['timestamp','user','minorEdit','pageLength','editDiff','comment','tags','entireEdit'])
i=0
for line in lines:
        entireEdit = line
        #split via brackets
        tokens = re.split('\(|\)',line)
        timestamp = tokens[2].split('?')[0]
        timestamp = pd.to_datetime(timestamp)
        #split by ?
        user = tokens[2].split('?')[1:]
        user = ''.join(user)
        user = user[1:-1]
        m = tokens[4]
        m= m.split('.')[0]
        m= True if (m=='? m ') else False
        byte = tokens[5].split(' ')[0]
        byte = int(byte.replace(',',''))
        change = tokens[7]
        change = int(change.replace(',',''))
        comment = re.split('\. \. ',line)[-1]
        tagComment = comment.split('(undo)')
        comment = tagComment[0][1:-2]
        #undo is not at the end
        if (len(tagComment)>1):
            tag = tagComment[1][2:-2]
            tag = tag.split(':')
            if (len(tag)>1):
                tag = tag[1][1:]
                tag = tag.split(',')
            if (tag[0]==''):
                tag =[]
        else:
            tag=''

        df.loc[i]=[timestamp,user,m,byte,change,comment,tag,entireEdit]
        i+=1

print (df.shape)

(1338, 8)


In [7]:
contributions = df['user'].value_counts()
blue = '#2E18F4'
red = '#FB412C' 
additions = pd.DataFrame(columns=['timestamp','user','size','color','editDiff','comment'])
deletions = pd.DataFrame(columns=['timestamp','user','size','color','editDiff','comment'])
users = contributions.index.tolist()
users = list(reversed(users))
maxDiff = max(df['editDiff'])
for row in df.iterrows():
    size = abs(row[1]['editDiff'] * 94 / maxDiff) + 6
    if(row[1]['editDiff'] < 0):
        deletions.loc[len(deletions)] = [row[1]['timestamp'],row[1]['user'],size,red,row[1]['editDiff'],row[1]['comment']]
    else:
        additions.loc[len(additions)] = [row[1]['timestamp'],row[1]['user'],size,blue,row[1]['editDiff'],row[1]['comment']]
        
blueSource = ColumnDataSource(data=dict(
    x=additions['timestamp'],
    y=additions['user'],
    size = additions['size'], 
    color = additions['color'],
    editDiff = additions['editDiff'],
    comment = additions['comment']
))
redSource = ColumnDataSource(data=dict(
    x=deletions['timestamp'],
    y=deletions['user'],
    size = deletions['size'], 
    color = deletions['color'],
    editDiff = deletions['editDiff'],
    comment = deletions['comment']
))
p = figure(plot_width=1000, plot_height=3500, x_axis_type="datetime", y_range=users,x_axis_location="above",title='Wiki edits over time',
           tools = [TapTool(),BoxZoomTool(), PanTool(),WheelZoomTool(), UndoTool(), RedoTool(), ResetTool(), ZoomInTool(), ZoomOutTool()])
p.add_layout(DatetimeAxis(), 'below')
p.yaxis.major_label_text_font_size = "6pt"

code = """
    var data = source.data,
    selected = source.selected['1d']['indices'],
    select_inds = selected[0];
    date = new Date(data['x'][select_inds])
    name = "<p><b>User: </b>"+data['y'][select_inds]+"</p>"
    timestamp = "<p><b>Timestamp: </b>"+date+"</p>"
    editDiff = "<p><b>Bytes Changed: </b>"+data['editDiff'][select_inds]+" Bytes</p>"
    comment = "<p><b>Comment: </b>"+data['comment'][select_inds]+"</p>"
    div.text = name+timestamp+editDiff+comment
"""

div = Div(text="",
width=400, height=100)


blueSource.callback = CustomJS(args={'source':blueSource, 'div':div},code=code)
redSource.callback = CustomJS(args={'source':redSource, 'div':div},code=code)

p.circle('x','y', size='size', color=blue, alpha=0.3, source=blueSource, legend='Bytes added', line_width=0,
         selection_fill_alpha=1,selection_fill_color=blue, nonselection_fill_color=blue,selection_line_color=blue)
p.circle('x','y', size='size',color=red,alpha=0.3,source=redSource, legend = 'Bytes removed', line_width=0,
         selection_fill_alpha=1,selection_fill_color=red, nonselection_fill_color=red,selection_line_color=red)
p.legend.click_policy="hide"

heading = Div(text = '<h3>Details of the selected point</h3>',width=400,height=20)




output_file('wiki-edits.html')


show(Row(p,Column(heading,div)))

**Conclusion:** This graph is good for initial data analysis. It does a good job of representing the edits over time and including other properties such as bytes changed, bytes removed and bytes added. The most active users are plotted at the top of the graph while the less active users are plotted at the bottom of the graph. On clicking the points, the details of the point are displayed on the right of the graph. This graph can be reused for other wikipedia edits data as well as other data such as twitter. 